In [1]:
print('This notebook references the following tutorials:')
print()
print(' * building and saving an analysis tree')
print(' * loading and browsing an analysis tree')
print()

This notebook references the following tutorials:

 * building and saving an analysis tree
 * loading and browsing an analysis tree



In [2]:
print('------------------------------------')
print('Building and saving an analysis tree')
print('------------------------------------')
print()

from tramway.helper import *

# preliminary: if trajectory_file does not exist, generate it
from tutorial import *
load_default_trajectories()

# load trajectory_file
trajectories = load_xyt(trajectory_file)

# the (trans-)location data is the top node of the analysis tree
analysis_tree = Analyses(trajectories)

print('analysis tree with translocation data:')
print()
print(analysis_tree)
#<Analyses <class 'pandas.core.frame.DataFrame'>>
print('-------------------------------------')
print()

# partition the locations into space cells
hexagonal_cells = tessellate(trajectories, 'hexagon')

# the partition is a first analysis; let's append it below trajs with label 'hexagonal mesh'
analysis_tree['hexagonal mesh'] = hexagonal_cells

# the two lines above could be combined the following way:
#hexagonal_cells = tessellate(analysis_tree, 'hexagon', label='hexagonal mesh')

print('analysis tree with first partition:')
print()
print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
print('----------------------------------------------------------------------')
print()

# make another partition
random_cells = tessellate(trajectories, 'random', cell_count=100)

# ...and add it to the analysis_tree below trajs with label 'random mesh'
analysis_tree['random mesh'] = random_cells

# similarly, the two lines above could be combined the following way:
#random_cells = tessellate(analysis_tree, 'random', cell_count=100, label='random mesh')

print('analysis tree with second partition:')
print()
print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
#	'random mesh' <class 'tramway.tessellation.base.CellStats'>
print('----------------------------------------------------------------------')
print()

# infer in D mode with random_cells
random_maps = infer(random_cells, 'D', sigma=0.01) # sigma = positioning error

# add the inferred maps below 'random mesh' with label 'D'
analysis_tree['random mesh']['D'] = random_maps

# again, the two lines above could be combined the following way:
#random_maps = infer(analysis_tree, 'D', sigma=0.01, input_label='random mesh', output_label='D')

print('analysis tree with maps for the second partition:')
print()
print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
#	'random mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>
print('----------------------------------------------------------------------')
print()

# infer in D mode with snr extensions (for the bayes_factors plugin) and hexagonal_cells
hexagonal_maps = infer(hexagonal_cells, 'd.conj_prior', sigma=0.01)

# add the inferred maps below 'hexagonal mesh' with label 'D' as well
analysis_tree['hexagonal mesh']['D'] = hexagonal_maps

print('analysis tree with maps for the first partition:')
print()
print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>
#	'random mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>
print('----------------------------------------------------------------------')
print()

# compute the Bayes factor on these latter "maps"
bayes_factors = infer(analysis_tree, 'bayes_factor', sigma=0.01, input_label=('hexagonal mesh', 'D'), output_label='Bayes factors')
# here, the analysis tree is readily updated

print('analysis tree with Bayes factor maps:')
print()
print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>
#			'Bayes factors' <class 'tramway.inference.base.Maps'>
#	'random mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>
print('-----------------------------------------------------------------------------')
print()

# save the analyses
save_rwa('my_analyses.rwa', analysis_tree)


------------------------------------
Building and saving an analysis tree
------------------------------------

analysis tree with translocation data:

<Analyses <class 'pandas.core.frame.DataFrame'>>
-------------------------------------

analysis tree with first partition:

<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
----------------------------------------------------------------------

analysis tree with second partition:

<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
	'random mesh' <class 'tramway.tessellation.base.CellStats'>
----------------------------------------------------------------------



  0%|          | 0/33 [00:00<?, ?it/s]

analysis tree with maps for the second partition:

<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
	'random mesh' <class 'tramway.tessellation.base.CellStats'>
		'D' <class 'tramway.inference.base.Maps'>
----------------------------------------------------------------------

analysis tree with maps for the first partition:

<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
		'D' <class 'tramway.inference.base.Maps'>
	'random mesh' <class 'tramway.tessellation.base.CellStats'>
		'D' <class 'tramway.inference.base.Maps'>
----------------------------------------------------------------------



 52%|█████▏    | 17/33 [00:00<00:00, 80.41it/s]/usr/lib/python3/dist-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)
 67%|██████▋   | 22/33 [00:03<00:01,  6.15it/s]


analysis tree with Bayes factor maps:

<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
		'D' <class 'tramway.inference.base.Maps'>
			'Bayes factors' <class 'tramway.inference.base.Maps'>
	'random mesh' <class 'tramway.tessellation.base.CellStats'>
		'D' <class 'tramway.inference.base.Maps'>
-----------------------------------------------------------------------------

overwrite file 'my_analyses.rwa': [N/y] y


In [3]:
print('-------------------------------------')
print('Loading and browsing an analysis tree')
print('-------------------------------------')
print()

from tramway.core.hdf5 import load_rwa
# or if you do not mind to load many modules:
#from tramway.helper import load_rwa

# load
analysis_tree = load_rwa('my_analyses.rwa')

# print the tree structure
print('loaded analysis tree:')
print()
print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>
#			'Bayes factors' <class 'tramway.inference.base.Maps'>
#	'random mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>
print('-----------------------------------------------------------------------------')
print()

# retrieve some data
trajs = analysis_tree.data
hexagonal_cells = analysis_tree['hexagonal mesh'].data
hexagonal_maps = analysis_tree['hexagonal mesh']['D'].data
#...

# print a subtree
print("subtree from the 'random mesh' node:")
print()
print(analysis_tree['random mesh'])
#<class 'tramway.tessellation.base.CellStats'>
#	'D' <class 'tramway.inference.base.Maps'>
print('-------------------------------------------------')
print()

# print some data
print("'random mesh' partition:")
print()
print(analysis_tree['random mesh'].data)
#location_count:  None
#@time_window:    duration:           1.0
#                 shift:              1.0
#                 time_dimension:     True
#@upper_bound:    [ 1.1  1.1]
#cell_index:      <class 'numpy.ndarray'>
#@partition:      metric:             euclidean
#                 knn:                10
#@method:         random
#bounding_box:    None
#@cell_count:     100
#points:          <class 'pandas.core.frame.DataFrame'>
#tessellation:    <class 'tramway.tessellation.random.RandomMesh'>
#@lower_bound:    [-0.1 -0.1]
#number_of_cells: 100
#@tessellation:   cell_count:         100
#                 avg_probability:    0.033927056827820185
#                 avg_location_count: 80
print('-----------------------------------------------------------------')
print()

print("'random mesh'->'D' maps:")
print()
print(analysis_tree['random mesh']['D'].data)
#mode:      d
#maps:      <class 'pandas.core.frame.DataFrame'>
#sigma:     0.01
#runtime:   0.11431741714477539
#variables: ['diffusivity']
print('------------------------------------------------')
print()


-------------------------------------
Loading and browsing an analysis tree
-------------------------------------

loaded analysis tree:

<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
		'D' <class 'tramway.inference.base.Maps'>
			'Bayes factors' <class 'tramway.inference.base.Maps'>
	'random mesh' <class 'tramway.tessellation.base.CellStats'>
		'D' <class 'tramway.inference.base.Maps'>
-----------------------------------------------------------------------------

subtree from the 'random mesh' node:

<class 'tramway.tessellation.base.CellStats'>
	'D' <class 'tramway.inference.base.Maps'>
-------------------------------------------------

'random mesh' partition:

tessellation:    <class 'tramway.tessellation.random.RandomMesh'>
points:          <class 'pandas.core.frame.DataFrame'>
cell_index:      <class 'numpy.ndarray'>
location_count:  None
number_of_cells: 69
bounding_box:    None
@cell_count:     100
@method:         random
